<a href="https://colab.research.google.com/github/joberiai/Devour-Hope/blob/master/practica8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Práctica 8. Visión por computador II

En esta práctica vamos a crear un clasificador capaz de reconocer dígitos manuscritos, para ello utilizaremos el dataset [MNIST](http://yann.lecun.com/exdb/mnist/) que ya hemos utilizado en prácticas anteriores.

Recordad los 5 pasos de un clasificador de imágenes:
1. Preparar el dataset.
2. Separar en conjuntos de entrenamiento, test y validación.
3. Extraer descriptores.
4. Entrenar el modelo.
5. Evaluar el modelo.

En muchas ocasiones los pasos 2 y 3 son intercambiables, y es lo que haremos en esta práctica.

## 0. Carga del dataset y descripción de las imágenes

Para cargar el dataset y describir las imágenes vamos a utilizar las clases `CargaYDescribe` (definida en el fichero CargaYDescribe.py de la carpeta p10) y la clase `Descriptor` junto con sus hijas (definidas en el fichero Descriptor.py también dentro de la carpeta p10). Deberás descargar esta carpeta y descomprimirla utilizando el siguiente comando.



In [1]:
!wget www.unirioja.es/cu/joheras/ia/p10/p10.zip -O p10.zip
!unzip p10.zip

--2024-04-20 16:14:41--  http://www.unirioja.es/cu/joheras/ia/p10/p10.zip
Resolving www.unirioja.es (www.unirioja.es)... 193.146.238.61
Connecting to www.unirioja.es (www.unirioja.es)|193.146.238.61|:80... connected.
HTTP request sent, awaiting response... 302 Object moved
Location: https://www.unirioja.es:443/cu/joheras/ia/p10/p10.zip [following]
--2024-04-20 16:14:42--  https://www.unirioja.es/cu/joheras/ia/p10/p10.zip
Connecting to www.unirioja.es (www.unirioja.es)|193.146.238.61|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /cu/joheras/ia/p10/p10.zip [following]
--2024-04-20 16:14:43--  https://www.unirioja.es/cu/joheras/ia/p10/p10.zip
Reusing existing connection to www.unirioja.es:443.
HTTP request sent, awaiting response... 200 No headers, assuming HTTP/0.9
Length: unspecified
Saving to: ‘p10.zip’

p10.zip                 [            <=>     ]   1.60K  --.-KB/s    in 9.8s    

2024-04-20 16:14:53 (168 B/s) - ‘p10.zip’ saved [1636]

Archive:  p10.

Si abres el fichero Descriptor.py (puedes abrir dicho fichero pulsando en la pestaña con el símbolo ">" que aparece a la izquierda de la pantalla y luego pulsando en la pestaña Files), verás la definición de la clase `Descriptor` y de dos clases hijas. La principal característica de la clase `Descriptor` es que tiene un método llamado `describe` que sirve para describir una imagen. Este método es abstracto y lo proporcionan las clases hijas.

##### Pregunta

¿Qué hacen los métodos ``describe`` de las clases RawImage y Projection?



*   El método 'describe' de la clase RawImage primer comprueba si la longitud del 'shape' de la imagen pasada es de 3, es decir, que tiene canales y, por tanto, está a color. Si esto se cumple, cambia el color de la imagen a la escala de grises. Para acabar devuelve la propia imagen aplanada, es decir, un array con sus píxeles.
*   El método 'describe' de la clase Projection cambia la imagen a escala de grises en el caso de que esté a color, al igual que el de RawImage. Después, suma el valor de todos los píxeles de todas las filas y columnas de la imagen y los añade a un array, que es el parámetro de salida de la función.



Si ahora abres el fichero CargaYDescribe.py verás la definición de la clase `CargaYDescribe`. Esta clase permite cargar y describir las imágenes de un directorio que está organizado mediante carpetas donde cada carpeta contiene
las imágenes de una clase.

El constructor de esta clase recibe dos parámetros, un objeto de la clase `Descriptor`, lo que nos permite cambiar fácilmente el método que usamos para describir las imágenes (esta es una técnica llamada inyección de dependencias que ya hemos utilizado alguna vez), y un path que es donde se encuentran las carpetas con las imágenes de cada clase.

Definimos una variable donde almacenamos el path donde se encuentran nuestra imágenes. De nuevo, es necesario descargar la carpeta.

In [2]:
%%capture
!wget www.unirioja.es/cu/joheras/ia/p10/digitos.zip
!unzip digitos.zip

In [3]:
dataset = "digitos/"

Para este ejemplo vamos a utilizar como descriptor el RawImage.

In [4]:
from p10.Descriptor import *
from p10.CargaYDescribe import *

descriptor = Projection()
cargaDatos = CargaYDescribe(dataset,descriptor)
(datos,etiquetas) = cargaDatos.cargaYdescribe()

##### Ejercicio

Comprueba que puedes cambiar de descriptor y que todo sigue funcionando correctamente.


In [5]:
descriptor = RawImage()
cargaDatos = CargaYDescribe(dataset,descriptor)
(datos, etiquetas) = cargaDatos.cargaYdescribe()

## 1. Separando el conjunto de entrenamiento y el de test

##### Ejercicio

Utiliza la función train_test_split, vista en las prácticas de aprendizaje automático, para partir el dataset en la siguiente proporción: 75% entrenamiento, 25% test. Almacena la partición en las variables:
`trainData`, `testData`, `trainLabels`, `testLabels`. Utiliza como random_state el valor 84 para así obtener siempre los mismos resultados.

In [6]:
from sklearn.model_selection import train_test_split
import numpy as np

(trainData, testData, trainLabels, testLabels) = train_test_split(datos, etiquetas, test_size = 0.25, random_state = 84)

## 2. Entrenando el modelo

Una vez que hemos partido el banco de imágenes, podemos entrenar distintos modelos como vimos en prácticas anteriores.

**Ejercicio**

Utiliza KNN y aplica validación cruzada para fijar los hiperparámetros.

In [7]:
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier

scores = []

for k in range(1, 11):
  modelKNN = KNeighborsClassifier(n_neighbors = k)
  resultados = cross_val_score(modelKNN, trainData, trainLabels, cv = 10, scoring = 'accuracy')
  scores.append(resultados.mean())

print(scores)
nVecinos = scores.index(np.max(scores)) + 1
print("El número de vecinos debe ser", nVecinos)

[0.9145833333333334, 0.8859375, 0.9005208333333334, 0.90625, 0.9005208333333334, 0.8994791666666668, 0.8973958333333332, 0.8927083333333332, 0.8927083333333332, 0.8880208333333334]
El número de vecinos debe ser 1


**Ejercicio**

Una vez hayas encontrado los mejores hiperparámetros almacena y entrena el modelo.

In [8]:
modelKNN = KNeighborsClassifier(n_neighbors = nVecinos)
modelKNN.fit(trainData, trainLabels)

KNeighborsClassifier(n_neighbors=1)

##### Ejercicio

Utiliza otro de los algoritmos de clasificación vistos en las prácticas anteriores para crear un nuevo modelo.

In [9]:
from sklearn.tree import DecisionTreeClassifier

modelArbol = DecisionTreeClassifier(random_state = 84)
modelArbol.fit(trainData, trainLabels)

DecisionTreeClassifier(random_state=84)

## 3. Evaluando el modelo

##### Ejercicio
Una vez tienes tu modelo construido evalualo utilizando la función `classification_report` de la librería sklearn.

In [10]:
from sklearn.metrics import classification_report

predKNN = modelKNN.predict(testData)
print(classification_report(y_true = testLabels, y_pred = predKNN))

              precision    recall  f1-score   support

           0       0.94      1.00      0.97        66
           1       0.80      0.98      0.88        42
           2       0.94      0.84      0.89        69
           3       0.84      0.89      0.87        66
           4       0.93      0.86      0.90        66
           5       0.89      0.89      0.89        74
           6       0.97      0.99      0.98        71
           7       0.92      0.93      0.92        58
           8       0.98      0.72      0.83        58
           9       0.85      0.94      0.89        70

    accuracy                           0.90       640
   macro avg       0.91      0.91      0.90       640
weighted avg       0.91      0.90      0.90       640



##### Ejercicio

A continuación muestra la matriz de confusión para ver dónde se producen los fallos.

In [11]:
from sklearn.metrics import confusion_matrix

print("MATRIZ DE CONFUSIÓN USANDO KNN")
print(confusion_matrix(testLabels, predKNN))

MATRIZ DE CONFUSIÓN USANDO KNN
[[66  0  0  0  0  0  0  0  0  0]
 [ 0 41  0  0  0  1  0  0  0  0]
 [ 2  2 58  2  0  0  0  4  0  1]
 [ 0  1  1 59  0  3  0  1  1  0]
 [ 0  1  0  0 57  0  0  0  0  8]
 [ 1  1  0  4  0 66  1  0  0  1]
 [ 1  0  0  0  0  0 70  0  0  0]
 [ 0  1  1  0  0  0  0 54  0  2]
 [ 0  4  2  4  1  4  1  0 42  0]
 [ 0  0  0  1  3  0  0  0  0 66]]


*Árbol de decisión*

In [12]:
predArbol = modelArbol.predict(testData)
print(classification_report(y_true = testLabels, y_pred = predArbol))

              precision    recall  f1-score   support

           0       0.88      0.76      0.81        66
           1       0.79      0.79      0.79        42
           2       0.65      0.59      0.62        69
           3       0.73      0.67      0.70        66
           4       0.64      0.67      0.65        66
           5       0.68      0.64      0.66        74
           6       0.74      0.83      0.78        71
           7       0.71      0.83      0.76        58
           8       0.48      0.53      0.50        58
           9       0.64      0.61      0.63        70

    accuracy                           0.69       640
   macro avg       0.69      0.69      0.69       640
weighted avg       0.69      0.69      0.69       640



In [13]:
print("MATRIZ DE CONFUSIÓN USANDO ÁRBOL DE DECISIÓN")
print(confusion_matrix(testLabels, predArbol))

MATRIZ DE CONFUSIÓN USANDO ÁRBOL DE DECISIÓN
[[50  0  2  3  2  2  2  0  4  1]
 [ 0 33  0  1  0  1  2  3  2  0]
 [ 2  1 41  1  3  4  4  6  4  3]
 [ 1  1  1 44  0  6  1  2  7  3]
 [ 0  2  2  2 44  1  2  3  2  8]
 [ 1  1  1  5  5 47  2  1  6  5]
 [ 1  1  3  0  0  2 59  1  4  0]
 [ 0  0  5  0  1  2  0 48  1  1]
 [ 2  2  7  1  4  1  6  1 31  3]
 [ 0  1  1  3 10  3  2  3  4 43]]


##### Ejercicio

Con el modelo KNN somos capaces de conseguir una precisión del 92%, aunque no está mal, se puede mejorar. Intenta construir un modelo que mejore estos resultados, puedes probar también a cambiar de descriptor.

In [14]:
criterios = ["gini", "entropy", "log_loss"]
maxFeatures = ["sqrt", "log2", None]

In [15]:
from sklearn.ensemble import RandomForestClassifier

scoresArbol = [0]

for k in range(2, 21):
  modelBosque = RandomForestClassifier(n_estimators = 200, min_samples_split = k, bootstrap = False)
  resultadosArbol = cross_val_score(modelKNN, trainData, trainLabels, cv = 10, scoring = 'accuracy')
  scoresArbol.append(resultadosArbol.mean())

print(scoresArbol)
nMinsamplesSplit = scoresArbol.index(np.max(scoresArbol)) + 1
print("El número de vecinos debe ser", nMinsamplesSplit)

[0, 0.9145833333333334, 0.9145833333333334, 0.9145833333333334, 0.9145833333333334, 0.9145833333333334, 0.9145833333333334, 0.9145833333333334, 0.9145833333333334, 0.9145833333333334, 0.9145833333333334, 0.9145833333333334, 0.9145833333333334, 0.9145833333333334, 0.9145833333333334, 0.9145833333333334, 0.9145833333333334, 0.9145833333333334, 0.9145833333333334, 0.9145833333333334]
El número de vecinos debe ser 2


In [16]:
modelBosque = RandomForestClassifier(n_estimators = 1000, min_samples_split = nMinsamplesSplit, bootstrap = False)
modelBosque.fit(trainData, trainLabels)

RandomForestClassifier(bootstrap=False, n_estimators=1000)

In [17]:
predBosque = modelBosque.predict(testData)
print(classification_report(y_true = testLabels, y_pred = predBosque))

              precision    recall  f1-score   support

           0       0.89      0.95      0.92        66
           1       0.93      0.98      0.95        42
           2       0.94      0.88      0.91        69
           3       0.94      0.92      0.93        66
           4       0.95      0.95      0.95        66
           5       0.99      0.89      0.94        74
           6       0.93      1.00      0.97        71
           7       0.92      0.95      0.93        58
           8       0.89      0.83      0.86        58
           9       0.89      0.91      0.90        70

    accuracy                           0.93       640
   macro avg       0.93      0.93      0.93       640
weighted avg       0.93      0.93      0.93       640



Obtenemos una precisión del 93% con un árbol aleatorio

## 4. Guardando y cargando el modelo

El proceso de entrenar un modelo puede ser muy costoso, por lo que conviene guardar el modelo para poder recuperarlo posteriormente. Para ello utilizamos la librería `pickle`.

In [18]:
import pickle

La siguiente instrucción guarda el modelo en el fichero modelo.sav.

In [19]:
pickle.dump(modelKNN, open('modelo.sav', 'wb'))

La siguiente instrucción carga el modelo del fichero modelo.sav.

In [20]:
loaded_model = pickle.load(open('modelo.sav', 'rb'))

Ahora podemos usar el modelo igual que antes.

In [21]:
predictions = loaded_model.predict(testData)
print(classification_report(testLabels, predictions))

              precision    recall  f1-score   support

           0       0.94      1.00      0.97        66
           1       0.80      0.98      0.88        42
           2       0.94      0.84      0.89        69
           3       0.84      0.89      0.87        66
           4       0.93      0.86      0.90        66
           5       0.89      0.89      0.89        74
           6       0.97      0.99      0.98        71
           7       0.92      0.93      0.92        58
           8       0.98      0.72      0.83        58
           9       0.85      0.94      0.89        70

    accuracy                           0.90       640
   macro avg       0.91      0.91      0.90       640
weighted avg       0.91      0.90      0.90       640



Para descargar el fichero desde Colab a nuestro ordenador, hay que ejecutar las siguientes instrucciones (esta funcionalidad solo está disponible usando como navegador Google Chrome).

In [22]:
from google.colab import files
files.download('modelo.sav')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

##### Ejercicio

Guarda tu mejor modelo en el fichero `mimodelo.sav`.

In [23]:
# Guardamos y cargamos el modelo en el fichero mimodelo.sav
pickle.dump(modelBosque, open('mimodelo.sav', 'wb'))
loaded_model = pickle.load(open('mimodelo.sav', 'rb'))

predictions = loaded_model.predict(testData)
print(classification_report(testLabels, predictions))

              precision    recall  f1-score   support

           0       0.89      0.95      0.92        66
           1       0.93      0.98      0.95        42
           2       0.94      0.88      0.91        69
           3       0.94      0.92      0.93        66
           4       0.95      0.95      0.95        66
           5       0.99      0.89      0.94        74
           6       0.93      1.00      0.97        71
           7       0.92      0.95      0.93        58
           8       0.89      0.83      0.86        58
           9       0.89      0.91      0.90        70

    accuracy                           0.93       640
   macro avg       0.93      0.93      0.93       640
weighted avg       0.93      0.93      0.93       640



###### Ejercicio

Realizar un estudio estadístico utilizando el paquete StatisticalAnalisis visto en la práctica 6.


In [24]:
!pip install StatisticalAnalysis
from StatisticalAnalysis import compare_methods

  Preparing metadata (setup.py) ... done
  Created wheel for StatisticalAnalysis: filename=StatisticalAnalysis-0.0.5-py2.py3-none-any.whl size=13098 sha256=b4fffa979f647db978a135aa486fbcb9094e31bf34b6f665c94c9619a187e435
  Stored in directory: /root/.cache/pip/wheels/8c/db/95/26b1f6f0da09912e26583c42371b4ac0d4fd1a8348a8636b6b
Successfully built StatisticalAnalysis


In [43]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.linear_model import Perceptron
from sklearn.neural_network import MLPClassifier

from scipy.stats import randint as sp_randint

# Árbol de decisión
clfTree = DecisionTreeClassifier(random_state=84)
param_distTree = {"min_samples_split": sp_randint(3, 30)}
# SVM
clfSVC = SVC(random_state=84)
param_distSVC = {'C': [1, 10, 100, 1000], 'gamma': [0.001, 0.0001],'kernel': ['rbf'], 'class_weight':['balanced', None]}
# KNN
clfKNN = KNeighborsClassifier()
param_distKNN = {'n_neighbors':sp_randint(3, 30)}
# Regresión logística
clfLR = LogisticRegression(random_state=84)
param_distLR = {'C': [0.1,0.5,1, 10, 100, 1000]}
# Red neuronal
clfMLP = MLPClassifier(random_state=84)
param_distMLP = {'activation': ['identity', 'logistic', 'tanh', 'relu'], 'solver': ['lbfgs', 'sgd', 'adam'],
                 'alpha': [0.0001, 0.001, 0.01, 0.1, 1], 'learning_rate': ['constant', 'invscaling', 'adaptive'],
                 'hidden_layer_sizes': [(5,2), (3,3,3), (5,3,2), (5,4,3,2)],
                 'momentum': [0.9, 0.95, 0.99]}
# Bosque aleatorio

In [44]:
listAlgorithms = [clfTree, clfSVC, clfKNN, clfLR, clfMLP, clfBosque]
listParams = [param_distTree, param_distSVC, param_distKNN, param_distLR, param_distMLP, param_distBosque]
listNames = ["Arbol", "SVM", "KNN", "LR", "MLP", "Bosque"]

In [42]:
compare_methods(datos, etiquetas, listAlgorithms, listParams, listNames, metric = 'auroc')

TypeError: only integer scalar arrays can be converted to a scalar index

##### Ejercicio

En la práctica 5 se proporcionó un fichero que explicaba cómo utilizar la librería `keras` para construir un modelo capaz de predecir los dígitos manuscritos. Utiliza dicha funcionalidad para construir un modelo de predicción.

##### Ejercicio

El último ejercicio adicional que se propone consiste en combinar la funcionalidad de contornos vista en la práctica 7 y el clasificador de identificación de dígitos.

Comenzamos cargando las librerías y funcionalidades necesarias.

In [25]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import imutils
%matplotlib inline

In [29]:
def mostrarImagen(image):
    if len(image.shape)==3:
        img2 = image[:,:,::-1]
        plt.imshow(img2)
        plt.show()
    else:
        img2 = image
        plt.imshow(img2,cmap='gray')
        plt.show()

Cargamos la imagen en la cual queremos leer los dígitos escritos y la mostramos.

In [30]:
!wget www.unirioja.es/cu/joheras/ia/p10/imagen1.png
image = cv2.imread("imagen1.png")
mostrarImagen(image)

--2024-04-20 16:35:42--  http://www.unirioja.es/cu/joheras/ia/p10/imagen1.png
Resolving www.unirioja.es (www.unirioja.es)... 193.146.238.61
Connecting to www.unirioja.es (www.unirioja.es)|193.146.238.61|:80... connected.
HTTP request sent, awaiting response... 302 Object moved
Location: https://www.unirioja.es:443/cu/joheras/ia/p10/imagen1.png [following]
--2024-04-20 16:35:43--  https://www.unirioja.es/cu/joheras/ia/p10/imagen1.png
Connecting to www.unirioja.es (www.unirioja.es)|193.146.238.61|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /cu/joheras/ia/p10/imagen1.png [following]
--2024-04-20 16:35:44--  https://www.unirioja.es/cu/joheras/ia/p10/imagen1.png
Reusing existing connection to www.unirioja.es:443.
HTTP request sent, awaiting response... 200 No headers, assuming HTTP/0.9
Length: unspecified
Saving to: ‘imagen1.png.2’

imagen1.png.2           [               <=>  ] 311.46K  --.-KB/s    in 9.8s    

2024-04-20 16:35:54 (31.8 KB/s) - ‘imagen1.p

AttributeError: 'NoneType' object has no attribute 'shape'

Convierte la imagen a escala de grises.

Aplica una umbralización con valor de umbral 80.

Busca los cortornos de la imagen, y para cada uno de ellos encuentra el bounding box que lo contiene.

Utilizando el bounding box que contiene al contorno, predice el número contenido utilizando el modelo que hayas definido y muestra el resultado en la imagen. Debes tener en cuenta varias cosas, las imágenes del dataset tienen tamaño 28x28, por lo que es posible que tengas que reescalar tu imagen, además en las imágenes el digito aparece en blanco con fondo negro, mientras que en la imagen está justo al revés, por lo que tendrás que invertir los colores.

Guarda este fichero con tus soluciones a los distintos ejercicios usando la opción **"Save in Github..."**.